In [1]:
from langchain.memory import ConversationBufferMemory

In [2]:
memory = ConversationBufferMemory()
memory.save_context({"input": "hi"}, {"output": "whats up"})

In [3]:
memory.load_memory_variables({})

{'history': 'Human: hi\nAI: whats up'}

In [4]:
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})

In [5]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'), AIMessage(content='whats up')]}

---


In [6]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from operator import itemgetter

memory = ConversationBufferMemory(return_messages=True)

model = ChatOpenAI(temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Act as a chatbot that helps users with their queries."),
        MessagesPlaceholder(variable_name="history"), # The history of the conversation
        ("human", "{input}"),
    ]
)
chain = (
    {
        "input": lambda x: x["input"],
        "history": RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [7]:
inputs = {"input": "Hi my name is James!"}
result = chain.invoke(inputs)
memory.save_context(inputs, {"outputs": result})

In [8]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi my name is James!'),
  AIMessage(content='Hello James! How can I assist you today?')]}

In [9]:
inputs = {"input": "What is my name?"}
second_result = chain.invoke(inputs)
print(second_result)

Your name is James.
